# Imports

In [29]:
# Torch
import torch
import torch.nn as nn
import torchvision.datasets as dsets
from torchvision.utils import make_grid
import torchvision.transforms as transforms

# Wandb
import wandb

# Python
import random as rand

# Numpy
import numpy as np

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Seeding
torch.manual_seed(24)

# CPU/GPU device

In [30]:
def to_device(data, device):
    """
        Move tensor(s) to chosen device
    """
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """
        Wrap a dataloader to move data to a device
    """
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """
            Yield a batch of data after moving it to device
        """
        for b in self.dl: 
            yield to_device(b, self.device)
            
    def __len__(self):
        """
            Number of batches
        """
        return len(self.dl)

In [31]:
# Choose device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
print("Device: ", device)

Device:  cpu


# WANDB

In [33]:
# Define sweep
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'validation_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [13, 20, 40, 60, 80, 100]
        },
        'learning_rate': {
            'values': [1e-2, 3e-2, 1e-4, 3e-4, 1e-6, 3e-6]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'adagrad', 'adadelta', 'rmsprop']
        },
        'activation': {
            'values': ['tanh', 'relu', 'sigmoid']
        },
        'pool1': {
            'values': ['maxPool', 'avgPool']
        },
        'pool2': {
            'values': ['maxPool', 'avgPool']
        },
        'out_channels1': {
            'values': [6, 16, 32]
        },
        'out_channels2': {
            'values': [6, 16, 32]
        }
    }
}

In [11]:
# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="Assignment-4")

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /Users/dailand10/.netrc


Create sweep with ID: 97juurev
Sweep URL: https://wandb.ai/itaouil/Assignment-4/sweeps/97juurev


# Datasets

### Load

In [34]:
# Transform
transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load dataset
dataset = dsets.CIFAR10('./data', train=True, download=True, transform=transform)
test_dataset = dsets.CIFAR10('./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
# Labels
CLASSES = dataset.classes

In [36]:
len(CLASSES), CLASSES

(10,
 ['airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'])

### Preparation

In [37]:
# Create train and validation
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [38]:
print("Validation set lenght: ", len(val_dataset))
print("Train set length: ", len(train_dataset))

Validation set lenght:  5000
Train set length:  45000


In [39]:
# Create dataloaders
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, num_workers=4, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [40]:
# Move dataloader to device
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

### Parameters

In [41]:
# Params
batch_size = 100
val_size = 5000
train_size = len(dataset) - val_size

# Model

### Helper Functions

In [42]:
# Accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Evaluation
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Training
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 20,
        'learning_rate': 1e-2,
        'optimizer': 'adam',
        'activation': 'tanh',
        'pool1': 'maxPool',
        'pool2': 'maxPool',
        'out_channels1': '6',
        'out_channels2': '6'
    }
    
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config
    config = wandb.config

    # Model
    model = CIFAR10Model(config)
    model = to_device(model, device) 
    
    # Choose optimizer based on sweep
    # and add L2 regularizer
    optimizer = None
    if config.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), config.learning_rate, weight_decay=0.001)
    elif config.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), config.learning_rate, weight_decay=0.001)
    elif config.optimizer == 'adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), config.learning_rate, weight_decay=0.001)
    elif config.optimizer == 'adadelta':
        optimizer = torch.optim.Adadelta(model.parameters(), config.learning_rate, weight_decay=0.001)
    else:
        optimizer = torch.optim.RMSprop(model.parameters(), config.learning_rate, weight_decay=0.001)
    
    for epoch in range(config.epochs):
        
        # Cumulative loss
        closs = 0
        
        # Training Phase 
        for batch_idx,batch in enumerate(train_loader):
            # Feed forward
            loss = model.training_step(batch)
            
            # Compute gradient
            loss.backward()
            
            # Add loss to cumulative
            closs += loss.item()
            
            # Backprop
            optimizer.step()
            
            # Clean gradients
            optimizer.zero_grad()
            
            # Log wandb (batch loss)
            wandb.log({"batch_loss": loss.item()})
        
        # Log epoch loss (train)
        wandb.log({"train_loss": closs/len(train_loader)})
        
        # Log validation loss
        wandb.log({"validation_loss":evaluate(model, val_loader)['val_loss']})

        # Log validation accuracy
        wandb.log({"validation_accuracy":evaluate(model, val_loader)['val_acc']})
    
    # Save model
    torch.save(model, f"model_{config.optimizer}_{config.activation}_{config.learning_rate}_{config.epochs}.th")

### Model Class

In [43]:
class CIFAR10Model(nn.Module):
    def __init__(self, config):
        # Init
        super().__init__()
        
        # 1st CNN layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=config.out_channels1, kernel_size=5, stride=1, padding=0)
        
        # Pooling 1
        self.pool1 = nn.MaxPool2d(kernel_size=2) if config.pool1 == "maxPool" else nn.AvgPool2d(kernel_size=2)
        
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=config.out_channels1, out_channels=config.out_channels2, kernel_size=5, stride=1, padding=0)
        
        # Pooling 2
        self.pool2 = nn.MaxPool2d(kernel_size=2) if config.pool2 == "maxPool" else nn.AvgPool2d(kernel_size=2)
        
        # FC1
        self.fc1 = nn.Linear(config.out_channels2 * 4 * 4, 120)
        
        # FC2
        self.fc2 = nn.Linear(120, 84)
        
        # FC3
        self.fc3 = nn.Linear(84, 10)
    
    def activation(self, input_):
        # Output after activation
        output = None
        
        # Apply activation function
        if config.activation == "relu":
            output = torch.relu(input_)
        elif config.activation == "tanh":
            output = torch.tanh(input_)
        else:
            output = torch.sigmoid(input_)
                    
        return output 
        
    def forward(self, x):
        # First pass
        out = self.activation(self.conv1(x))
        out = self.pool1(out)
        
        # Second conv pass
        out = self.activation(self.conv2(out))
        out = self.pool2(out)

        # Flatten
        out = out.view(out.size(0), -1)
 
        # Linear function (readout)
        out = self.activation(self.fc1(out))
        out = self.activation(self.fc2(out))
        out = self.fc3(out)
         
        return out
    
    def training_step(self, batch):
        # Unpack batch
        images, labels = batch
        
        # Run forward pass
        out = self.forward(images)
        
        # Compute and return loss
        return nn.CrossEntropyLoss()(out, labels)
    
    def validation_step(self, batch):
        # Unpack batch
        images, labels = batch
        
        # Run forward pass
        out = self.forward(images)
        
        # Compute loss
        loss = nn.CrossEntropyLoss()(out, labels)
        
        # Compute accuracy
        acc = accuracy(out, labels)
        
        return {'val_loss': loss, 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

# Training

In [18]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 8ef9coam with config:
wandb: 	activation: tanh
wandb: 	epochs: 80
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: adam
wandb: Currently logged in as: itaouil (use `wandb login --relogin` to force relogin)
/home/user/taouil/cvl/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


batch_loss,0.34714
_step,14319
_runtime,212
_timestamp,1607016985
train_loss,0.39902
validation_loss,1.91345
validation_accuracy,0.51451


batch_loss,█▇█▆▆▇▆▆▆▆▆▆▅▅▄▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▃▂▂▂▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
validation_loss,▇▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇█
validation_accuracy,▁▃▄▆▆▆▇▇▇▇▇▇▇█▇█████▇██████▇████▇▇███▇▇█


wandb: Agent Starting Run: ydjg3mkv with config:
wandb: 	activation: tanh
wandb: 	epochs: 80
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: sgd


batch_loss,2.01759
_step,14319
_runtime,208
_timestamp,1607017197
train_loss,2.02873
validation_loss,2.03341
validation_accuracy,0.28213


...

batch_loss,█▆▅▄▅▃▄▄▄▃▂▃▄▄▂▃▄▃▃▂▂▃▄▃▃▃▃▂▂▂▁▄▄▂▁▂▃▃▄▃
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▆▅▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████


wandb: Agent Starting Run: 95tqyd12 with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 80
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: adadelta


wandb: Ctrl + C detected. Stopping sweep.
Exception in thread Thread-7613:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/user/taouil/cvl/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/user/taouil/cvl/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address,

batch_loss,2.31568
_step,11571
_runtime,171
_timestamp,1607023615
train_loss,2.30695
validation_loss,2.30791
validation_accuracy,0.10622


batch_loss,▅▃█▄▃▃▂▁▄▃▂▂▄▁▂▁▂▂▂▃▃▃▃▂▂▁▂▂▃▄▂▃▂▃▂▂▃▂▄▂
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▆▄▄▃▂▂▁▂▁▂▁▁▂▁▂▁▁▂▂▁▂▂▂▂▁▂▂▂▂▁▂▁▁▁▂▂▂▁▂
validation_accuracy,▆▅▄▄▅▅▃▅▁▃█▄▃▄▅▄▄▄▂▃▁▄▅▃▇▇▄▃▄█▅▃▆▄▆▃▄▅▇▇


# Comparison

The below graph shows the comparison between the different sweeps that consisted of different optimizers and activation functions:

<img src="comparison.png">

we now show the confusion matrix of the best model training within the 80 epochs, which is the one with the following parameters:

- Adagrad
- 80 epochs
- relu
- 0.0003

In [18]:
# Load model
model = torch.load("model_adagrad_relu_0.0003_80.th")
model = to_device(model, "cpu")

cm = np.zeros((10, 10), int)
for i in range(len(test_dataset)):
    data = test_dataset[i][0]
    label = test_dataset[i][1]
    data = data.view(1,data.shape[0],data.shape[1],data.shape[2])
    predict = model.forward(data)
    target = label
    cm[predict.argmax(), target] += 1

print(cm)

[[521  23  73  27  63  22   6  35  79  26]
 [ 45 632  35  37  16  18  17  19  69 169]
 [ 43   3 309  67 147  94  53  44  11   4]
 [ 31  17 100 316  66 215  79  62  24  37]
 [ 25   7 114  33 337  44  79  70  12   9]
 [ 16  18  82 202  58 351  49  73  13  11]
 [ 31  16 157 147 158 104 640  44  14  23]
 [ 25  19  66  50  86  76  22 539  12  33]
 [190  78  38  37  37  34  22  22 679  66]
 [ 73 187  26  84  32  42  33  92  87 622]]


finally we evaluate the same model on the test_loader to get the loass and accuracy:

In [20]:
model = to_device(model, device)
evaluate(model, test_loader)

{'val_loss': 1.412893533706665, 'val_acc': 0.49335938692092896}

The validation accuracy is not the best, although the capacity of the model was increased, and the validation loss is quite high still, so further training would have definetely improved the model to breach the 50% accuracy.